# core

> This module contains all the core functions used in the library.

In [ ]:
#| default_exp core

In [ ]:
#| export
import logging
import os

from rich.logging import RichHandler

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(RichHandler(rich_tracebacks=True))

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from datasets import load_dataset
from squeakily.filter import check_char_repetition, check_flagged_words
from squeakily.clean import remove_empty_lines, normalize_whitespace

logging.getLogger("datasets").setLevel(logging.ERROR)

In [ ]:
#| export
class Pipeline:
    """
    A pipeline is a collection of datasources and their associated transformations to be run.
    """
    def __init__(
        self,
        datasources # The datasources to be run
    ):
        self.datasources = datasources
    
    def run(
        self,
        cleaning_first=False # Whether to run the cleaning transformations first
    ):
        """
        Run the pipeline.
        """
        for datasource in self.datasources:
            dataset = datasource["dataset"]
            column = datasource["columns"][0]
            logger.info(f"Running datasource: {dataset.builder_name}")
            if cleaning_first:
                for c in datasource["cleaners"]:
                    logger.info(f"Running cleaner: {c.__name__} on {column}")
                    dataset = dataset.map(
                        lambda x: {column: c(x[column])},
                        num_proc=os.cpu_count(),
                    )
                for f in datasource["filters"]:
                    logger.info(f"Running filter: {f.__name__} on {column}")
                    dataset = dataset.filter(lambda x: f(x[column]))
            else:
                for f in datasource["filters"]:
                    logger.info(f"Running filter: {f.__name__} on {column}")
                    dataset = dataset.filter(lambda x: f(x[column]))
                for c in datasource["cleaners"]:
                    logger.info(f"Running cleaner: {c.__name__} on {column}")
                    dataset = dataset.map(
                        lambda x: {column: c(x[column])},
                        num_proc=os.cpu_count(),
                    )

In [ ]:
ds = load_dataset("wikitext", "wikitext-103-v1", split="train[:10%]")
datasources = [
    {
        "dataset": ds,
        "columns": ["text"],
        "filters": [check_char_repetition, check_flagged_words],
        "cleaners": [remove_empty_lines, normalize_whitespace],
    },
    # ...
]

pipeline = Pipeline(datasources)
pipeline.run()

[11/06/22 21:12:02] INFO     Running datasource: wikitext                                          ]8;id=537018;file:///tmp/ipykernel_280307/3163439817.py\3163439817.py]8;;\:]8;id=411647;file:///tmp/ipykernel_280307/3163439817.py#22\22]8;;\

                    INFO     Running filter: check_char_repetition on text                         ]8;id=577052;file:///tmp/ipykernel_280307/3163439817.py\3163439817.py]8;;\:]8;id=150348;file:///tmp/ipykernel_280307/3163439817.py#35\35]8;;\

                    INFO     Running filter: check_flagged_words on text                           ]8;id=808825;file:///tmp/ipykernel_280307/3163439817.py\3163439817.py]8;;\:]8;id=821956;file:///tmp/ipykernel_280307/3163439817.py#35\35]8;;\

                    INFO     Running cleaner: remove_empty_lines on text                           ]8;id=672553;file:///tmp/ipykernel_280307/3163439817.py\3163439817.py]8;;\:]8;id=786979;file:///tmp/ipykernel_280307/3163439817.py#38\38]8;;\

                    INFO     Running cleaner: normalize_whitespace on text                         ]8;id=296654;file:///tmp/ipykernel_280307/3163439817.py\3163439817.py]8;;\:]8;id=778363;file:///tmp/ipykernel_280307/3163439817.py#38\38]8;;\

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()